In [1]:
import pandas as pd
import scraper_functions as func
import reshaping_functions as shape
import json

### Collecting the links for the vote pages

In [ ]:
first_page = "https://www.ft.dk/dokumenter/dokumentlister/afstemninger?startDate=20020201&endDate=20220315&pageSize=200&totalNumberOfRecords=8967"
current_page = func.get_soup_page(first_page)
next_page = True
vote_page_links = []
pages_viewed = 0

while next_page == True:
    vote_links = func.get_vote_links(current_page)
    links_set = set(vote_links)
    vote_links = [val for val in links_set]
    vote_links.sort(reverse=True)
    
    #saving the link pages.
    vote_page_links += [val for val in links_set]
    
    if func.exists_next_page:
        current_page = func.get_next_page(current_page)
        pages_viewed += 1
        if pages_viewed % 10 == 0: 
            print(pages_viewed, end="")
    else: 
        next_page = False
        print("Finoto!")

    
    #process_bar
    print("|", end="")


In [ ]:
save = False
if save == True:
    with open("vote_pages_links.json", "w") as fp:
        json.dump(vote_page_links, fp)

In [2]:
with open("vote_pages_links.json", "r") as fp:
    vp = json.load(fp)

print("Vote pages fetched:", len(vp))



Vote pages fetched: 8931


In [ ]:
dict = {'vote_id': [], 'description':[], 'votes':[], 
        'votes_by_party':[], 'vote_caller':[], 'url':[]}
pages_processed = 0

for i in range(len(vp)):
    
    try:
        url = vp[i]
        vote_page = func.get_soup_page(url)
        
        #id
        vote_id = func.get_vote_id(vote_page)
        
        try:
            description_page = func.get_description_page(vote_page)
            description = func.get_vote_info(description_page)
        except:
            description = "no description"
            
        #votes by politician
        votes = func.get_votes(vote_page)
        
        try:
            vote_caller = func.get_vote_caller(description_page)
        except:
            vote_caller = None
            
        #votes aggregated by party
        votes_by_party = func.get_votes_by_party(vote_page)    
        
        dict['vote_id'].append(vote_id)
        dict['description'].append(description)
        dict['votes_by_party'].append(votes_by_party)
        dict['vote_caller'].append(vote_caller)
        dict['votes'].append(votes)
        dict['url'].append(url)
        
        #process_bar
        pages_processed += 1
        print("|", end="")
        if pages_processed % 50 == 0: print(pages_processed)
        #if pages_processed == 10: break
        
    except:
        dict['vote_id'].append("no data")
        dict['description'].append("no data")
        dict['votes_by_party'].append("no data")
        dict['vote_caller'].append("no data")
        dict['votes'].append("note data")
        dict['url'].append(url)
        

In [13]:
#df = pd.DataFrame(dict)
# df.to_pickle("votes_scraped_over_night2.pkl")
# df.to_csv("votes_scraped_over_night2.csv")


In [51]:
df = pd.read_pickle("votes_scraped_over_night.pkl")


In [29]:
print("Number of rows missing the description for the vote:", len(df[df.description == 'no description']))
print("number of rows missing data altogether for dataset:", len(df[df.vote_id == "no data"]))

print(df[df.vote_id == "no data"]['url'][359])
print(vp[360])
df[df.vote_id == "no data"]


df[df.url.duplicated() == True]

Number of rows missing the description for the vote: 229
number of rows missing data altogether for dataset: 22
https://www.ft.dk//samling/20211/afstemning/55.htm
https://www.ft.dk//samling/20201/afstemning/703.htm


,vote_id,description,votes,votes_by_party,vote_caller,url
6139,"Afstemning nr. 373, 2009-10","[Fremsat:, 25-05-2010, Forslag til vedtagelse:...","{'politician': ['Anders Samuelsen', 'Anita Chr...","{'parties': ['Venstre (V)', 'Socialdemokratiet...","[Hanne Agersnap (udpeget af SF) (SF), Jørgen S...",https://www.ft.dk//samling/20091/afstemning/37...


In [22]:
missing_urls = []

for url in df[df['description'] == "no data"]['url']:
    missing_urls.append(url)

In [24]:
dict = {'vote_id': [], 'description':[], 'votes':[], 
        'votes_by_party':[], 'vote_caller':[], 'url':[]}
pages_processed = 0

for i in range(len(missing_urls)):
    
    try:
        url = missing_urls[i]
        vote_page = func.get_soup_page(url)
        
        #id
        vote_id = func.get_vote_id(vote_page)
        
        try:
            description_page = func.get_description_page(vote_page)
            description = func.get_vote_info(description_page)
        except:
            description = "no description"
            
        #votes by politician
        votes = func.get_votes(vote_page)
        
        try:
            vote_caller = func.get_vote_caller(description_page)
        except:
            vote_caller = None
            
        #votes aggregated by party
        votes_by_party = func.get_votes_by_party(vote_page)    
        
        dict['vote_id'].append(vote_id)
        dict['description'].append(description)
        dict['votes_by_party'].append(votes_by_party)
        dict['vote_caller'].append(vote_caller)
        dict['votes'].append(votes)
        dict['url'].append(url)
        
        #process_bar
        pages_processed += 1
        print("|", end="")
        if pages_processed % 50 == 0: print(pages_processed)
        #if pages_processed == 10: break
        
    except:
        dict['vote_id'].append(None)
        dict['description'].append(None)
        dict['votes_by_party'].append(None)
        dict['vote_caller'].append(None)
        dict['votes'].append(None)
        dict['url'].append(url)

||||||||||||||||||||

In [ ]:
df.drop(index=df[df['vote_id']=='no data'].index, inplace=True)

df_missing_entries = pd.DataFrame(dict)
df.append(df_missing_entries)

In [53]:
df.to_pickle("votes_scraped_over_night_v2.pkl")